Use cases are being developed to demonstrate the usage of PicoQuant and for use in benchmarking and profiling activities.

## Quantum Fourier transfrom use case

The Quantum Fourier Transform (QFT) is at the heart of many quantum algorithms. For this use case a simple implementation from [https://arxiv.org/abs/quant-ph/0201067](https://arxiv.org/abs/quant-ph/0201067) has been implemented. Here the gate count grows approximately with the square of the number of qubits. A methond called `create_qft_circuit` can be used to construct a qiskit circuit object for this circuit for a given number of qubits. For example to create a QFT circuit for 3 qubits one can use:

In [ ]:
using PicoQuant

qft_circ = create_qft_circuit(3)

qft_circ.draw()

We can convert this circuit to a tensor network circuit using the `convert_qiskit_circ_to_network` method as 

In [ ]:
tn = convert_qiskit_circ_to_network(qft_circ)
plot(tn)

We then add an all zero input state and use a full wavefunction contraction plan to contract the network.

In [ ]:
add_input!(tn, "000")
full_wavefunction_contraction!(tn, "vector")
psi = load_tensor_data(tn, :result)

This gives the final transformed state. We can verify this is the correct answer by carrying out an inverse fourier transform on the input vector. Note that we switch the endianness before and after applying the IFFT so the conventions match.

In [ ]:
using FFTW

function switch_endianness(vec)
    n = convert(Int, log2(length(vec)))
    vec = reshape(vec, Tuple([2 for _ in 1:n]))
    vec = permutedims(vec, [i for i in n:-1:1])
    reshape(vec, 2^n)
end

function get_ft_with_ifft(input_state)    
    output_state = ifft(switch_endianness(input_state))
    output_state /= sqrt(sum(output_state .* conj(output_state)))
    switch_endianness(output_state)
end

input_state = zeros(ComplexF64, 8)
input_state[1] = 1.
get_ft_with_ifft(input_state)

We can test with a less trivial input state by using a state preparation circuit to prepare a less trivial state. 

In [ ]:
prep_circ = create_simple_preparation_circuit(3, 2)
prep_circ.draw()

We combine this with the QFT circuit to apply the QFT to the prepared state and then run the full wavefunction contraction to get the final state

In [ ]:
combined_circ = prep_circ.compose(qft_circ)
tn = convert_qiskit_circ_to_network(combined_circ)
add_input!(tn, "000")
full_wavefunction_contraction!(tn, "vector")
psi = load_tensor_data(tn, :result)

To check that the output is correct we get the output of just the state preparation circuit and apply the IFFT with appropriate normalization (using the get_ft_with_ifft defined above).

In [ ]:
tn = convert_qiskit_circ_to_network(prep_circ)
add_input!(tn, "000")
full_wavefunction_contraction!(tn, "vector")
psi_input = load_tensor_data(tn, :result)
ref_output = get_ft_with_ifft(psi_input)

We see that these match.

## Grover search use case

Grover's search is a seminal algorithm in quantum information processing, demonstrating $O\left(\sqrt{m}\right)$ scaling for finding an item in a database search, compared to the optimal $O({m})$ using classical methods (https://arxiv.org/abs/quant-ph/9605043). Here we implement a use-case using a 5-qubit example with circuit depth that grows as $O\left(n^2\right)$ using the full state-vector implementation, and another with 11 qubits (split as 6 compute, 4 auxiliary, 1 result ), with circuit depth growing as $O\left(n\right)$ for the MPS example. These use an implemented $n$-controlled unitary library internally, which largely dictates the scaling of the examples.

We begin by importing the *QuantExQASM* package, which is used to implement high-order methods for circuit generation. The resulting circuit is then convertable to OpenQASM, or directly to PicoQuant gate calls, depending upon the required functionality.

In [ ]:
using Pkg; Pkg.add(PackageSpec(url="https://github.com/ICHEC/QuantExQASM.jl"))

We choose the 5 qubit example initially, and assume no auxiliary qubits for the circuit. The function `create_grover_circuit` creates the required intermediate gates, generates the circuit, and exports it to a Qiskit circuit, for tensor network generation. 

For the chosen `bit_pattern` the resulting amplitude will be amplified following execution of the circuit.

In [ ]:
using PicoQuant

In [ ]:
module CreateGroverCircuit
    import QuantExQASM

    export create_circuit

    function create_circuit(num_qubits, use_aux_qubits, bit_pattern)
        QuantExQASM.Algorithms.create_grover_circuit(num_qubits, use_aux_qubits, bit_pattern)
    end 
end

In [ ]:
num_qubits = 5
use_aux_qubits = false
bit_pattern = 10
grover_cct = CreateGroverCircuit.create_circuit(num_qubits, use_aux_qubits, bit_pattern)
tn_grover = convert_qiskit_circ_to_network(grover_cct)
plot(tn_grover)

Following the QFT example, we can obtain the wavefunction amplitudes by contracting the network. The index of the largest amplitude (-1) will match that of the chosen `bit_pattern` value.

In [ ]:
add_input!(tn_grover, "0"^num_qubits)
psi_node = full_wavefunction_contraction!(tn_grover, "vector")
psi = load_tensor_data(tn_grover, psi_node)

In [ ]:
println("Index $(argmax(abs.(psi).^2)-1) is equal to bit-pattern $(bit_pattern) with amplitude $( (abs.(psi).^2)[bit_pattern+1] )")

In [ ]:
import Plots
Plots.plot(0:(2^num_qubits-1), abs.(psi).^2, label="Amplitudes", markershape=:auto)
ground_truth = zeros(length(psi))
ground_truth[bit_pattern + 1] = 1.0
Plots.plot!(0:(2^num_qubits-1), ground_truth, label="Ground truth", markershape=:auto, alpha=0.7)

We can also use an optimised $n$-controlled unitary implementation to allow better scaling for larger circuits. This follows the implementation of https://arxiv.org/abs/quant-ph/9503016 which for an $n$-controlled unitary gate, we use an additional $n-2$ auxiliary qubits to reduce the overall depth. Here we demonstrate the example using 6 computational qubits, with an additional 4 for auxiliary (and 1 for the result).

We make use of the MPS backend of PicoQuant to perform this contraction.

In [ ]:
num_qubits = 11
use_aux_qubits = true
bit_pattern = 14

grover_cct_aux = CreateGroverCircuit.create_circuit(num_qubits, use_aux_qubits, bit_pattern)

tn_grover_aux = convert_qiskit_circ_to_network(grover_cct_aux, decompose=true, transpile=true)
plot(tn_grover_aux)

In [ ]:
add_input!(tn_grover_aux, "0"^num_qubits)
mps_nodes = contract_mps_tensor_network_circuit!(tn_grover_aux, max_bond=1)
calculate_mps_amplitudes!(tn_grover_aux, mps_nodes)
psi = load_tensor_data(tn_grover_aux, :result)
println("Index $(argmax(abs.(psi).^2)-1) is equal to bit-pattern $(bit_pattern) with amplitude $( (abs.(psi).^2)[bit_pattern+1] )")

For both examples, the largest index is where we expect, showing the state is amplified appropriately.